In [29]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix

In [30]:
# import "C:/Projects/public/XAI_Master/datasets/red1green0.csv"

red1green0 = pd.read_csv("C:/Projects/public/XAI_Master/datasets/red1green0.csv",
    header=0,          # Use first row as headers
    index_col=None,    # Don't use any column as index
    float_precision='high'  # For precise float values
    )
red0green1 = pd.read_csv("C:/Projects/public/XAI_Master/datasets/red0green1.csv",
    header=0,
    index_col=None,
    float_precision='high'
    )

# Merge and shuffle the two datasets
df = pd.concat([red1green0, red0green1])
df = df.sample(frac=1).reset_index(drop=True)
"""
df = red0green1
"""
x = df.drop(['target', 
            "agentX", "agentY",
            "redX", "redY",
            "greenX", "greenY"], axis=1)
print(x.shape)
y = df['target']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

(10000, 128)


In [31]:
model = LinearRegression()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.4f}")
print(f"R2 Score: {r2:.4f}")

feature_importance = pd.DataFrame({
    'feature': x.columns,
    'coefficient': model.coef_
})

print("\nTop 10 most important features by absolute coefficient value:")
print(feature_importance.reindex(feature_importance['coefficient'].abs().sort_values(ascending=False).index)[:10])

Mean Squared Error: 0.1504
R2 Score: 0.7538

Top 10 most important features by absolute coefficient value:
    feature   coefficient
68      a69 -4.605642e+09
39      a40  3.762482e+09
101    a102 -3.596363e+09
61      a62  3.091752e+09
16      a17  3.075428e+09
4        a5 -2.845484e+09
29      a30  2.691350e+09
125    a126  2.607024e+09
18      a19  2.434498e+09
44      a45  2.430724e+09


In [32]:
# Train decision tree
tree = DecisionTreeClassifier(random_state=42)
tree.fit(x_train, y_train)

# Make predictions
y_pred = tree.predict(x_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
total_correct = (y_test == y_pred).sum()
total_samples = len(y_test)

print(f"Total correct predictions: {total_correct} out of {total_samples}")
print(f"Accuracy: {accuracy:.2%}")

# Confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion matrix:")
print(conf_matrix)

Total correct predictions: 1568 out of 2000
Accuracy: 78.40%

Confusion matrix:
[[994  32   0]
 [  5 574   0]
 [  0 395   0]]


A baseline is established. Between one good red agent and one week green agent, the decision tree managed to predict 78.4% of the cases correctly based on only neuron activations from, fc1, the second last layer with 128 neurons. When also giving the tree the coordinates for the agent, red and greens chests, the tree performed much better, only failing at 7 of 2000 test cases.

My thoughts now is that I wonder how much the decision tree learns to identify the neural net and therefore knows if it is a good or bad neural net. I'm also thinking that I need many more neural nets to balance out this, especially very diffferent ones.
But probably, the next step is to try with a XAI method instead of activa
tions, since it is this comparison which is the most important part of the thesis. Maybe gradcam or shap. SHAP is probably good since it yield a map of what the agent "focuses" on. It is also a great baseline for later. So, in short, do shap, and more models.